In [1]:
import razorback as rb

In [2]:
def sensor(ats_file):
    header = rb.io.ats.read_ats_header(ats_file)
    chan = header['channel_type'].decode()
    stype = ''.join(c for c in header['sensor_type'].decode() if c.isprintable())
    snum = header['sensor_serial_number']
    sampling_rate = header['sampling_rate']
    x1, y1, z1 = header['x1'], header['y1'], header['z1']
    x2, y2, z2 = header['x2'], header['y2'], header['z2']
    L = ((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)**.5
    return chan, L, stype, snum, sampling_rate

In [3]:
def calibration(ats_file, name_converter=None):
    chan, L, stype, snum, sampling_rate = sensor(ats_file)
    if chan in ('Ex', 'Ey'):
        return L
    elif chan in ('Hx', 'Hy', 'Hz'):
        calib_name = f"{stype}{snum:03d}.txt"
        if name_converter:
            calib_name = name_converter.get(calib_name, calib_name)
        return rb.calibrations.metronix(calib_name, sampling_rate)
    raise Exception(f"Unknown channel name: {chan}")

In [4]:
import glob

files = glob.glob("data/site*/*/*.ats")
pattern = "*/site{site}/*/*_T{channel}_*.ats"
tag_template = "site{site}_{channel}"

name_converter = {
    'UNKN_H104.txt': 'MFS07104.txt',
    'UNKN_H105.txt': 'MFS07105.txt',
}

inv = rb.Inventory()
for fname, [tag] in rb.utils.tags_from_path(files, pattern, tag_template):
    calib = calibration(fname, name_converter)
    signal = rb.io.ats.load_ats([fname], [calib], lazy=True)
    inv.append(rb.SignalSet({tag:0}, signal))

In [5]:
len(inv)

26

In [6]:
inv.tags

{'site002_Ex',
 'site002_Ey',
 'site002_Hx',
 'site002_Hy',
 'site002_Hz',
 'site004_Ex',
 'site004_Ey',
 'site004_Hx',
 'site004_Hy',
 'site004_Hz',
 'site006_Ex',
 'site006_Ey',
 'site006_Hx',
 'site006_Hy',
 'site006_Hz',
 'site009_Ex',
 'site009_Ey',
 'site009_Hx',
 'site009_Hy',
 'site009_Hz',
 'site099_Hx',
 'site099_Hy',
 'site100_Ex',
 'site100_Ey',
 'site100_Hx',
 'site100_Hy'}

In [7]:
print(inv.filter('site002*').pack())

SignalSet: 5 channels, 1 run
tags: {'site002_Ex': (0,), 'site002_Ey': (1,), 
       'site002_Hx': (2,), 'site002_Hy': (3,), 
       'site002_Hz': (4,)}
----------  -------------------  -------------------
  sampling                start                 stop
       128  2016-04-28 08:20:00  2016-04-29 06:46:22
----------  -------------------  -------------------


In [8]:
print(inv.filter('site099*').pack())

SignalSet: 2 channels, 1 run
tags: {'site099_Hx': (0,), 'site099_Hy': (1,)}
----------  -------------------  -------------------
  sampling                start                 stop
       128  2016-04-28 19:00:00  2016-04-29 17:59:00
----------  -------------------  -------------------


In [9]:
print(inv.pack())

SignalSet: 26 channels, 1 run
tags: {'site002_Ex': (0,), 'site002_Ey': (1,), 
       'site002_Hx': (2,), 'site002_Hy': (3,), 
       'site002_Hz': (4,), 'site004_Ex': (5,), 
       'site004_Ey': (6,), 'site004_Hx': (7,), 
       'site004_Hy': (8,), 'site004_Hz': (9,), 
       'site006_Ex': (10,), 'site006_Ey': (11,), 
       'site006_Hx': (12,), 'site006_Hy': (13,), 
       'site006_Hz': (14,), 'site009_Ex': (15,), 
       'site009_Ey': (16,), 'site009_Hx': (17,), 
       'site009_Hy': (18,), 'site009_Hz': (19,), 
       'site099_Hx': (20,), 'site099_Hy': (21,), 
       'site100_Ex': (22,), 'site100_Ey': (23,), 
       'site100_Hx': (24,), 'site100_Hy': (25,)}
----------  -------------------  -------------------
  sampling                start                 stop
       128  2016-04-28 19:00:00  2016-04-29 04:00:05
----------  -------------------  -------------------


In [10]:
from itertools import chain

def prepare_signalset(inventory, local_site, remote_sites):
    patterns = (f"{e}*" for e in [local_site, *remote_sites])
    signalset = inventory.filter(*patterns).pack()
    tags = signalset.tags
    tags["E"] = tags[f"{local_site}_Ex"] + tags[f"{local_site}_Ey"]
    tags["B"] = tags[f"{local_site}_Hx"] + tags[f"{local_site}_Hy"]
    if remote_sites:
        remote_names = tags.filter(*chain(*(
            (f"{e}_Hx", f"{e}_Hy") for e in remote_sites
        )))
        tags["Bremote"] = sum((tags[n] for n in remote_names), ())
    return signalset

In [11]:
print(prepare_signalset(inv, 'site002', ['site100', 'site099']))

SignalSet: 11 channels, 1 run
tags: {'site002_Ex': (0,), 'site002_Ey': (1,), 
       'site002_Hx': (2,), 'site002_Hy': (3,), 
       'site002_Hz': (4,), 'site099_Hx': (5,), 
       'site099_Hy': (6,), 'site100_Ex': (7,), 
       'site100_Ey': (8,), 'site100_Hx': (9,), 
       'site100_Hy': (10,), 'B': (2, 3), 
       'E': (0, 1), 'Bremote': (10, 6, 5, 9)}
----------  -------------------  -------------------
  sampling                start                 stop
       128  2016-04-28 19:00:00  2016-04-29 06:46:22
----------  -------------------  -------------------


In [12]:
sig = prepare_signalset(inv, 'site002', [])
print(sig)

SignalSet: 5 channels, 1 run
tags: {'site002_Ex': (0,), 'site002_Ey': (1,), 
       'site002_Hx': (2,), 'site002_Hy': (3,), 
       'site002_Hz': (4,), 'B': (2, 3), 
       'E': (0, 1)}
----------  -------------------  -------------------
  sampling                start                 stop
       128  2016-04-28 08:20:00  2016-04-29 06:46:22
----------  -------------------  -------------------


In [13]:
rb.utils.impedance(sig, [10, 20, 30])

starting frequency 10
starting frequency 20
starting frequency 30


ImpedanceResult(impedance=array([[[-4.18992617e-06-4.13804525e-06j,
         -1.84279022e-06-1.59718126e-06j],
        [ 1.58132062e-05+1.07571335e-05j,
          4.28783119e-06+2.45357319e-06j]],

       [[-2.58332218e-06-5.92987407e-06j,
         -3.28932050e-06-3.40490004e-06j],
        [ 1.90845256e-05+2.24987199e-05j,
          2.86747376e-06+3.50877337e-06j]],

       [[-3.04219023e-06-2.77350932e-06j,
         -1.86932661e-07-1.89030199e-07j],
        [ 9.79103966e-06+7.84426644e-06j,
          4.21095006e-07+3.57852364e-07j]]]), invalid_time=[(array([], dtype=float64), array([], dtype=float64)), (array([], dtype=float64), array([], dtype=float64)), (array([], dtype=float64), array([], dtype=float64))], error=array([[[2.56056577e-08, 1.12961713e-08],
        [4.34611517e-08, 1.91732867e-08]],

       [[3.40400194e-08, 1.45057873e-08],
        [2.84216559e-08, 1.21115822e-08]],

       [[1.22844288e-08, 1.94635704e-09],
        [2.12291263e-08, 3.36356376e-09]]]))